## Getting Data

In [1]:
import os
import re
import csv
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn_crfsuite as crfsuite
from sklearn_crfsuite import metrics

## Importing dataset from a text file

In [26]:
with open('final.txt', encoding = "utf8") as file:
    lines = [line.rstrip() for line in file]

## Converting to list of list per article

In [3]:
main_lst=[]
temp_lst = []
no=1
for line in lines:
    if line=='':
        main_lst.append(temp_lst)
        temp_lst=[]
        print(f'Article: {no}')
        no = no+1
    else:
        temp_lst.append(line)

Article: 1
Article: 2
Article: 3
Article: 4
Article: 5
Article: 6
Article: 7
Article: 8
Article: 9
Article: 10
Article: 11
Article: 12
Article: 13
Article: 14
Article: 15
Article: 16
Article: 17
Article: 18
Article: 19
Article: 20
Article: 21
Article: 22
Article: 23
Article: 24
Article: 25
Article: 26
Article: 27
Article: 28
Article: 29
Article: 30
Article: 31
Article: 32
Article: 33
Article: 34
Article: 35
Article: 36
Article: 37
Article: 38
Article: 39
Article: 40
Article: 41
Article: 42
Article: 43
Article: 44
Article: 45
Article: 46
Article: 47
Article: 48
Article: 49
Article: 50
Article: 51
Article: 52
Article: 53
Article: 54
Article: 55
Article: 56
Article: 57
Article: 58
Article: 59
Article: 60
Article: 61
Article: 62
Article: 63
Article: 64
Article: 65
Article: 66
Article: 67
Article: 68
Article: 69
Article: 70
Article: 71
Article: 72
Article: 73
Article: 74
Article: 75
Article: 76
Article: 77
Article: 78
Article: 79
Article: 80
Article: 81
Article: 82
Article: 83
Article: 84
A

In [5]:
#testing for single line
#algo for converting to tuple
tuple(filter(None,[re.sub('[\s_]','',x) for x in main_lst[0][1].split(' ')]))

('pagsaka', '-X-', 'O')

## Converting to list of tuple per article

In [6]:
final_train=[]
for article in main_lst:
    temp_lst=[]
    for item in article:
        temp_lst.append(tuple(filter(None,[re.sub('[\s_]','',x) for x in item.split(' ')])))
    final_train.append(temp_lst)

In [7]:
len(final_train)

4257

In [8]:
#final output = final_train
final_train[0]

[("Aduna'y", '-X-', 'O'),
 ('pagsaka', '-X-', 'O'),
 ('sa', '-X-', 'O'),
 ('presyo', '-X-', 'O'),
 ('sa', '-X-', 'O'),
 ('lana', '-X-', 'O'),
 (',', '-X-', 'O'),
 ('epektibo', '-X-', 'O'),
 ('sa', '-X-', 'O'),
 ('Martes', '-X-', 'O'),
 (',', '-X-', 'O'),
 ('Hunyo', '-X-', 'O'),
 ('27', '-X-', 'O'),
 (',', '-X-', 'O'),
 ('2023', '-X-', 'O'),
 ('.', '-X-', 'O')]

In [9]:
#splitting
training_data = final_train[0:2980]
testing_data = final_train[2980:3831]
dev_data = final_train[3831:]

## Feature Extraction

In [10]:
def read_clusters(cluster_file):
    word2cluster = {}
    with open(cluster_file, encoding = "utf8") as i:
        for line in i:
            word, cluster = line.strip().split('\t')
            word2cluster[word] = cluster
    return word2cluster


def word2features(sent, i, word2cluster):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'word.cluster=%s' % word2cluster[word.lower()] if word.lower() in word2cluster else "0",
        'postag=' + postag
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1
        ])
    else:
        features.append('BOS')

    if i > 1: 
        word2 = sent[i-2][0]
        postag2 = sent[i-2][1]
        features.extend([
            '-2:word.lower=' + word2.lower(),
            '-2:word.istitle=%s' % word2.istitle(),
            '-2:word.isupper=%s' % word2.isupper(),
            '-2:postag=' + postag2
        ])        

        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1
        ])
    else:
        features.append('EOS')

    if i < len(sent)-2:
        word2 = sent[i+2][0]
        postag2 = sent[i+2][1]
        features.extend([
            '+2:word.lower=' + word2.lower(),
            '+2:word.istitle=%s' % word2.istitle(),
            '+2:word.isupper=%s' % word2.isupper(),
            '+2:postag=' + postag2
        ])

        
    return features


def sent2features(sent, word2cluster):
    return [word2features(sent, i, word2cluster) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

word2cluster = read_clusters("cebtenten.tsv")

In [11]:
sent2features(training_data[0], word2cluster)[0]

['bias',
 "word.lower=aduna'y",
 "word[-3:]=a'y",
 "word[-2:]='y",
 'word.isupper=False',
 'word.istitle=False',
 'word.isdigit=False',
 '0',
 'postag=-X-',
 'BOS',
 '+1:word.lower=pagsaka',
 '+1:word.istitle=False',
 '+1:word.isupper=False',
 '+1:postag=-X-',
 '+2:word.lower=sa',
 '+2:word.istitle=False',
 '+2:word.isupper=False',
 '+2:postag=-X-']

In [12]:
X_train = [sent2features(s, word2cluster) for s in training_data]
y_train = [sent2labels(s) for s in training_data]

X_dev = [sent2features(s, word2cluster) for s in dev_data]
y_dev = [sent2labels(s) for s in dev_data]

X_test = [sent2features(s, word2cluster) for s in testing_data]
y_test = [sent2labels(s) for s in testing_data]

## Training

In [13]:
crf = crfsuite.CRF(
    verbose='true',
    algorithm='lbfgs',
    max_iterations=100
)

try:
    crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
except AttributeError:
    pass
predictions = crf.predict(X_test)

loading training data to CRFsuite: 100%|█████████████████████████████████████████| 2980/2980 [00:02<00:00, 1134.28it/s]


loading dev data to CRFsuite: 100%|████████████████████████████████████████████████| 426/426 [00:00<00:00, 2337.79it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 123401
Seconds required: 0.639

L-BFGS optimization
c1: 0.000000
c2: 1.000000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.46  loss=98443.72 active=123401 precision=0.100  recall=0.111  F1=0.105  Acc(item/seq)=0.897 0.258  feature_norm=1.00
Iter 2   time=0.23  loss=96478.14 active=123401 precision=0.100  recall=0.111  F1=0.105  Acc(item/seq)=0.897 0.258  feature_norm=0.96
Iter 3   time=0.23  loss=91327.27 active=123401 precision=0.100  recall=0.111  F1=0.105  Acc(item/seq)=0.897 0.258  feature_norm=0.91
Iter 4   time=0.24  loss=87442.31 active=123401 precision=0.100  recall=0.111  F1=0.105  Acc(item/seq)=0.897 0.258  feature_norm=1.01
Iter 5   time=0.22  loss=68232.65 active=123

Iter 60  time=0.22  loss=14394.89 active=123401 precision=0.717  recall=0.646  F1=0.674  Acc(item/seq)=0.962 0.629  feature_norm=34.72
Iter 61  time=0.44  loss=14319.68 active=123401 precision=0.728  recall=0.672  F1=0.697  Acc(item/seq)=0.962 0.631  feature_norm=35.37
Iter 62  time=0.23  loss=14193.56 active=123401 precision=0.725  recall=0.657  F1=0.687  Acc(item/seq)=0.962 0.631  feature_norm=36.18
Iter 63  time=0.23  loss=14084.58 active=123401 precision=0.739  recall=0.656  F1=0.690  Acc(item/seq)=0.963 0.636  feature_norm=36.85
Iter 64  time=0.23  loss=13989.39 active=123401 precision=0.741  recall=0.653  F1=0.689  Acc(item/seq)=0.963 0.634  feature_norm=37.43
Iter 65  time=0.22  loss=13913.79 active=123401 precision=0.723  recall=0.639  F1=0.672  Acc(item/seq)=0.963 0.634  feature_norm=37.91
Iter 66  time=0.23  loss=13844.91 active=123401 precision=0.739  recall=0.660  F1=0.694  Acc(item/seq)=0.964 0.646  feature_norm=38.12
Iter 67  time=0.22  loss=13770.09 active=123401 precisi

In [14]:
import joblib
import os

OUTPUT_PATH = "./"
OUTPUT_FILE = "CebuaNER"

if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

joblib.dump(crf, os.path.join(OUTPUT_PATH, OUTPUT_FILE))

['./crf_model_ceb']

## Evaluation

In [23]:
crf = joblib.load(os.path.join(OUTPUT_PATH, "crf_model_ceb"))
y_pred = crf.predict(X_test)

example_sent = testing_data[0]

print("Sentence:", ' '.join(sent2tokens(example_sent)))
print("Predicted:", ' '.join(crf.predict([sent2features(example_sent, word2cluster)])[0]))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

Sentence: Ang Supreme Court En Banc nimando sa presiding judge sa RTC 52 sa Bohol nga mo-isyu og commitment order alang sa pagbalhog kang PB Member Boniel sa provincial jail sa Sugbo .
Predicted: O B-ORG I-ORG I-ORG I-ORG O O O O O B-ORG O O B-LOC O O O O O O O O O O B-PER I-PER O O O O B-LOC O
Correct:   O B-ORG I-ORG I-ORG I-ORG O O O O O B-ORG I-ORG O B-LOC O O O O O O O O O O O B-PER O O O O B-LOC O


In [16]:
labels = list(crf.classes_)
labels.remove("O")
y_pred = crf.predict(X_test)
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

from sklearn import metrics

# Flatten the y_test and y_pred sequences
y_test_flat = [item for sublist in y_test for item in sublist]
y_pred_flat = [item for sublist in y_pred for item in sublist]

# Print the classification report
print(metrics.classification_report(y_test_flat, y_pred_flat, digits=3))

              precision    recall  f1-score   support

       B-LOC      0.854     0.731     0.788       383
       B-ORG      0.825     0.558     0.665       312
     B-OTHER      0.528     0.308     0.389        91
       B-PER      0.859     0.895     0.877       524
       I-LOC      0.851     0.670     0.750       273
       I-ORG      0.835     0.736     0.782       420
     I-OTHER      0.462     0.505     0.482       109
       I-PER      0.852     0.917     0.883       264
           O      0.981     0.993     0.987     19729

    accuracy                          0.965     22105
   macro avg      0.783     0.701     0.734     22105
weighted avg      0.963     0.965     0.963     22105



## Finding the optimal parameters

In [18]:
import scipy
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite.metrics import flat_f1_score

crf = crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

f1_scorer = make_scorer(flat_f1_score, average='weighted', labels=labels)

rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)


Fitting 3 folds for each of 50 candidates, totalling 150 fits


RandomizedSearchCV(cv=3,
                   estimator=CRF(algorithm='lbfgs',
                                 all_possible_transitions=True,
                                 max_iterations=100),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000261FDF6FF50>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000261FDF6FB50>},
                   scoring=make_scorer(flat_f1_score, average=weighted, labels=['B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-OTHER', 'I-OTHER', 'B-ORG', 'I-ORG']),
                   verbose=1)

In [19]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.049493450726418434, 'c2': 0.21277148535244264}
best CV score: 0.7288294978289449
model size: 2.64M


In [20]:
from sklearn_crfsuite.metrics import flat_classification_report

best_crf = rs.best_estimator_
y_pred = best_crf.predict(X_test)
print(flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

       B-LOC      0.896     0.791     0.840       383
       I-LOC      0.830     0.714     0.768       273
       B-ORG      0.884     0.635     0.739       312
       I-ORG      0.860     0.731     0.790       420
     B-OTHER      0.650     0.429     0.517        91
     I-OTHER      0.596     0.596     0.596       109
       B-PER      0.879     0.914     0.896       524
       I-PER      0.865     0.943     0.902       264

   micro avg      0.851     0.772     0.810      2376
   macro avg      0.807     0.719     0.756      2376
weighted avg      0.850     0.772     0.805      2376

